<a href="https://colab.research.google.com/github/pranavpabbisetty/ETH-Prediction/blob/main/Ethereum_Price_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install prophet --no-cache-dir
!pip install yfinance --quiet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 KB 837.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 KB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 KB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 KB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 15.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly
import warnings

warnings.filterwarnings('ignore')

pd.options.display.float_format = '${:,.2f}'.format

In [ ]:
today = datetime.today().strftime('%Y-%m-%d')
start_date = '2016-01-01'

eth_df = yf.download('ETH-USD',start_date, today)

eth_df.tail()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-02-02 00:00:00+00:00,"$1,641.37","$1,704.46","$1,641.32","$1,643.24","$1,643.24",10558081069
2023-02-03 00:00:00+00:00,"$1,642.90","$1,670.70","$1,634.22","$1,664.75","$1,664.75",8169519805
2023-02-04 00:00:00+00:00,"$1,664.47","$1,690.10","$1,648.19","$1,667.06","$1,667.06",5843302512
2023-02-05 00:00:00+00:00,"$1,667.17","$1,671.77","$1,616.39","$1,631.65","$1,631.65",6926696531
2023-02-06 00:00:00+00:00,"$1,631.65","$1,653.72","$1,611.32","$1,616.25","$1,616.25",6919871886


In [ ]:
eth_df.isnull().sum()

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [ ]:
eth_df.columns

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [ ]:
eth_df.reset_index(inplace=True)
eth_df.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [ ]:
df = eth_df[["Date", "Open"]]

new_names = {
    "Date": "ds", 
    "Open": "y",
}

df.rename(columns=new_names, inplace=True)

In [ ]:
df.tail()

,ds,y
1911,2023-02-02 00:00:00+00:00,"$1,641.37"
1912,2023-02-03 00:00:00+00:00,"$1,642.90"
1913,2023-02-04 00:00:00+00:00,"$1,664.47"
1914,2023-02-05 00:00:00+00:00,"$1,667.17"
1915,2023-02-06 00:00:00+00:00,"$1,631.65"


In [ ]:
# plot the open price

x = df["ds"]
y = df["y"]

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y))

# Set title
fig.update_layout(
    title_text="Time series plot of Ethereum Open Price",
)

fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        ),
        rangeslider=dict(visible=True),
        type="date",
    )
)

In [ ]:
m = Prophet(
    seasonality_mode="multiplicative" 
)
df['ds'] = df['ds'].dt.tz_localize(None)
m.fit(df)

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmp21umlebm/j_0iribn.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp21umlebm/qjzej25t.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.8/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=41286', 'data', 'file=/tmp/tmp21umlebm/j_0iribn.json', 'init=/tmp/tmp21umlebm/qjzej25t.json', 'output', 'file=/tmp/tmp21umlebm/prophet_modelj15rf1e2/prophet_model-20230207210905.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
21:09:05 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
21:09:07 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


In [ ]:
future = m.make_future_dataframe(periods = 365)
future.tail()

,ds
2276,2024-02-02
2277,2024-02-03
2278,2024-02-04
2279,2024-02-05
2280,2024-02-06


In [ ]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
2276,2024-02-02,"$-1,306.70","$-2,611.04",$-29.79
2277,2024-02-03,"$-1,311.10","$-2,564.34",$57.19
2278,2024-02-04,"$-1,329.12","$-2,691.33",$-17.54
2279,2024-02-05,"$-1,345.98","$-2,709.58",$51.78
2280,2024-02-06,"$-1,358.57","$-2,734.51",$2.36


In [ ]:
next_day = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')

forecast[forecast['ds'] == next_day]['yhat'].item()

848.4782071924218

In [ ]:
plot_plotly(m, forecast)

In [ ]:
plot_components_plotly(m, forecast)